---
title: "Week 2: Data Wrangling & Feature Engineering for Finance"
subtitle: "INSTRUCTOR VERSION - Complete Solutions"
author: "Praveen Kumar"
date: 2025-10-08
week: 2
type: "instructor_notebook"
version: v1.0
---

# Week 2: Data Wrangling & Feature Engineering for Finance
## **INSTRUCTOR VERSION** 🎓

This notebook contains **complete solutions** for all exercises. Use this for:
- Preparing lecture demonstrations
- Providing detailed explanations to students
- Checking student work against reference implementations

⚠️ **CONFIDENTIAL**: Do not share this version with students.

In [ ]:
# INSTRUCTOR SETUP - Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Week 2: Feature Engineering for Financial Data")
print("🎓 INSTRUCTOR VERSION - All solutions included")
print("=" * 60)

## Section 1: Data Loading & Initial Exploration

**INSTRUCTOR NOTE**: This section demonstrates multiple data loading strategies and comprehensive data validation techniques.

In [ ]:
# INSTRUCTOR ONLY - Complete data loading with error handling
def load_financial_data_instructor(symbol="AAPL", period="2y", fallback=True):
    """
    INSTRUCTOR SOLUTION: Robust data loading with multiple fallback strategies
    """
    try:
        print(f"🔄 Attempting to download {symbol} data...")
        ticker = yf.Ticker(symbol)
        df = ticker.history(period=period)
        
        if df.empty:
            raise ValueError("Empty dataset returned")
            
        # Convert column names to lowercase for consistency
        df.columns = df.columns.str.lower()
        print(f"✅ Successfully loaded {len(df)} rows of {symbol} data")
        return df
        
    except Exception as e:
        print(f"❌ Error loading {symbol}: {e}")
        
        if fallback:
            print("🔄 Generating synthetic data for demonstration...")
            return generate_synthetic_data_instructor()
        else:
            raise

def generate_synthetic_data_instructor(n_days=500):
    """
    INSTRUCTOR SOLUTION: Generate realistic financial data with proper statistical properties
    """
    np.random.seed(42)  # Reproducible results
    
    # Generate dates
    end_date = datetime.now()
    start_date = end_date - timedelta(days=n_days)
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Generate price series with realistic properties
    initial_price = 150.0
    returns = np.random.normal(0.0005, 0.02, len(dates))  # Daily returns with slight positive drift
    
    # Add volatility clustering (GARCH-like behavior)
    vol = np.ones(len(dates)) * 0.02
    for i in range(1, len(dates)):
        vol[i] = 0.05 * vol[i-1] + 0.95 * abs(returns[i-1])
    
    returns = returns * vol
    prices = initial_price * np.exp(np.cumsum(returns))
    
    # Generate OHLC data
    highs = prices * (1 + np.abs(np.random.normal(0, 0.01, len(dates))))
    lows = prices * (1 - np.abs(np.random.normal(0, 0.01, len(dates))))
    opens = np.roll(prices, 1)
    opens[0] = initial_price
    
    # Generate volume with realistic patterns
    base_volume = 1000000
    volume = base_volume * (1 + np.random.exponential(0.5, len(dates)))
    
    df = pd.DataFrame({
        'open': opens,
        'high': highs,
        'low': lows,
        'close': prices,
        'volume': volume.astype(int)
    }, index=dates)
    
    print(f"✅ Generated {len(df)} days of synthetic data")
    return df

# Load data using instructor method
df = load_financial_data_instructor("AAPL", "2y")
print(f"\nDataset shape: {df.shape}")
print(f"Date range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
print("\nFirst 5 rows:")
print(df.head())

## Section 2: Data Cleaning & Preprocessing

**INSTRUCTOR NOTE**: Comprehensive data quality assessment and cleaning procedures for financial data.

In [ ]:
# INSTRUCTOR SOLUTION - Complete data cleaning pipeline
def comprehensive_data_cleaning_instructor(df):
    """
    INSTRUCTOR SOLUTION: Professional-grade data cleaning for financial data
    """
    print("🧹 COMPREHENSIVE DATA CLEANING")
    print("=" * 40)
    
    # Create a copy to avoid modifying original
    cleaned_df = df.copy()
    initial_rows = len(cleaned_df)
    
    # 1. Check for missing values
    missing_summary = cleaned_df.isnull().sum()
    print("📊 Missing Values Summary:")
    for col, missing in missing_summary.items():
        if missing > 0:
            pct = (missing / len(cleaned_df)) * 100
            print(f"  {col}: {missing} ({pct:.2f}%)")
    
    # 2. Handle missing values with forward fill (common for financial data)
    cleaned_df = cleaned_df.fillna(method='ffill').fillna(method='bfill')
    
    # 3. Validate OHLC relationships
    print("\n🔍 OHLC Validation:")
    invalid_high = (cleaned_df['high'] < cleaned_df[['open', 'close']].max(axis=1)).sum()
    invalid_low = (cleaned_df['low'] > cleaned_df[['open', 'close']].min(axis=1)).sum()
    
    print(f"  Invalid high prices: {invalid_high}")
    print(f"  Invalid low prices: {invalid_low}")
    
    # Fix invalid OHLC (adjust highs and lows)
    cleaned_df['high'] = np.maximum(cleaned_df['high'], 
                                   cleaned_df[['open', 'close']].max(axis=1))
    cleaned_df['low'] = np.minimum(cleaned_df['low'], 
                                  cleaned_df[['open', 'close']].min(axis=1))
    
    # 4. Detect and handle outliers using IQR method
    print("\n📈 Outlier Detection (Returns):")
    returns = cleaned_df['close'].pct_change()
    Q1 = returns.quantile(0.25)
    Q3 = returns.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 3 * IQR  # Using 3*IQR for conservative outlier detection
    upper_bound = Q3 + 3 * IQR
    
    outliers = ((returns < lower_bound) | (returns > upper_bound)) & (~returns.isna())
    print(f"  Outliers detected: {outliers.sum()} ({(outliers.sum()/len(returns))*100:.2f}%)")
    
    if outliers.sum() > 0:
        print(f"  Extreme returns range: {returns[outliers].min():.4f} to {returns[outliers].max():.4f}")
        # Cap outliers instead of removing (preserves data continuity)
        returns_capped = returns.clip(lower_bound, upper_bound)
        outliers_capped = outliers.sum()
        print(f"  Capped {outliers_capped} outlier returns")
    
    # 5. Check for suspicious volume patterns
    print("\n📊 Volume Analysis:")
    volume_outliers = (cleaned_df['volume'] > cleaned_df['volume'].quantile(0.99)).sum()
    zero_volume = (cleaned_df['volume'] == 0).sum()
    print(f"  High volume days (>99th percentile): {volume_outliers}")
    print(f"  Zero volume days: {zero_volume}")
    
    # 6. Final validation
    final_rows = len(cleaned_df)
    print(f"\n✅ Cleaning completed:")
    print(f"  Rows: {initial_rows} → {final_rows}")
    print(f"  No missing values: {cleaned_df.isnull().sum().sum() == 0}")
    
    return cleaned_df

# Apply comprehensive cleaning
cleaned_data = comprehensive_data_cleaning_instructor(df)
print("\nCleaned data sample:")
print(cleaned_data.head())

## Section 3: Basic Feature Engineering

**INSTRUCTOR NOTE**: This section covers fundamental financial features with proper implementation details and edge case handling.

In [ ]:
# INSTRUCTOR SOLUTION - Complete basic feature engineering
def engineer_basic_features_instructor(df):
    """
    INSTRUCTOR SOLUTION: Comprehensive basic feature engineering
    """
    print("⚙️ ENGINEERING BASIC FEATURES")
    print("=" * 40)
    
    # Create feature DataFrame
    features_df = df.copy()
    
    # 1. Returns (multiple types)
    print("📈 Calculating returns...")
    features_df['returns'] = df['close'].pct_change()
    features_df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    
    # Overnight returns (open vs previous close)
    features_df['overnight_returns'] = (df['open'] / df['close'].shift(1)) - 1
    
    # Intraday returns (close vs open)
    features_df['intraday_returns'] = (df['close'] / df['open']) - 1
    
    print(f"  Returns summary - Mean: {features_df['returns'].mean():.6f}, Std: {features_df['returns'].std():.6f}")
    
    # 2. Moving averages (multiple timeframes)
    print("📊 Calculating moving averages...")
    for window in [5, 10, 20, 50]:
        features_df[f'sma_{window}'] = df['close'].rolling(window=window).mean()
        features_df[f'ema_{window}'] = df['close'].ewm(span=window).mean()
    
    # Price relative to moving averages
    features_df['price_to_sma20'] = df['close'] / features_df['sma_20']
    features_df['price_to_sma50'] = df['close'] / features_df['sma_50']
    
    # 3. Volatility measures
    print("📉 Calculating volatility...")
    for window in [5, 10, 20, 30]:
        features_df[f'volatility_{window}'] = features_df['returns'].rolling(window=window).std()
        # Annualized volatility
        features_df[f'vol_annualized_{window}'] = features_df[f'volatility_{window}'] * np.sqrt(252)
    
    # Realized volatility (using high-low)
    features_df['realized_vol'] = np.log(df['high'] / df['low'])
    
    # 4. Volume features
    print("📊 Engineering volume features...")
    features_df['volume_sma_20'] = df['volume'].rolling(20).mean()
    features_df['volume_ratio'] = df['volume'] / features_df['volume_sma_20']
    
    # Volume-price trend
    features_df['vpt'] = (df['volume'] * ((df['close'] - df['close'].shift(1)) / df['close'].shift(1))).cumsum()
    
    # 5. Price gaps
    print("🔄 Calculating price gaps...")
    features_df['gap'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1)
    features_df['gap_up'] = (features_df['gap'] > 0.01).astype(int)  # Gap up > 1%
    features_df['gap_down'] = (features_df['gap'] < -0.01).astype(int)  # Gap down > 1%
    
    # 6. High-Low ranges
    print("📏 Calculating price ranges...")
    features_df['daily_range'] = (df['high'] - df['low']) / df['close']
    features_df['body_size'] = abs(df['close'] - df['open']) / df['close']
    features_df['upper_shadow'] = (df['high'] - np.maximum(df['open'], df['close'])) / df['close']
    features_df['lower_shadow'] = (np.minimum(df['open'], df['close']) - df['low']) / df['close']
    
    print(f"✅ Basic features engineered: {len([col for col in features_df.columns if col not in df.columns])} new features")
    return features_df

# Apply basic feature engineering
df_with_basic_features = engineer_basic_features_instructor(cleaned_data)

# Display feature summary
new_features = [col for col in df_with_basic_features.columns if col not in cleaned_data.columns]
print(f"\nNew features created ({len(new_features)}):")
for i, feature in enumerate(new_features, 1):
    print(f"{i:2d}. {feature}")

print(f"\nFeature DataFrame shape: {df_with_basic_features.shape}")

## Section 4: Advanced Technical Indicators

**INSTRUCTOR NOTE**: Implementation of sophisticated technical analysis indicators with proper parameter optimization and interpretation guidelines.

In [ ]:
# INSTRUCTOR SOLUTION - Complete technical indicators implementation
def calculate_advanced_indicators_instructor(df):
    """
    INSTRUCTOR SOLUTION: Professional technical indicators with optimized parameters
    """
    print("🔧 ADVANCED TECHNICAL INDICATORS")
    print("=" * 40)
    
    tech_df = df.copy()
    
    # 1. RSI with proper implementation
    print("📊 Calculating RSI...")
    def calculate_rsi_instructor(prices, window=14):
        delta = prices.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        # Use Wilder's smoothing (EMA with alpha = 1/window)
        avg_gain = gain.ewm(alpha=1/window, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/window, adjust=False).mean()
        
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    tech_df['rsi'] = calculate_rsi_instructor(df['close'])
    tech_df['rsi_overbought'] = (tech_df['rsi'] > 70).astype(int)
    tech_df['rsi_oversold'] = (tech_df['rsi'] < 30).astype(int)
    
    # 2. MACD with signal and histogram
    print("📈 Calculating MACD...")
    def calculate_macd_instructor(prices, fast=12, slow=26, signal=9):
        ema_fast = prices.ewm(span=fast).mean()
        ema_slow = prices.ewm(span=slow).mean()
        macd_line = ema_fast - ema_slow
        signal_line = macd_line.ewm(span=signal).mean()
        histogram = macd_line - signal_line
        return macd_line, signal_line, histogram
    
    tech_df['macd'], tech_df['macd_signal'], tech_df['macd_histogram'] = calculate_macd_instructor(df['close'])
    tech_df['macd_bullish'] = (tech_df['macd'] > tech_df['macd_signal']).astype(int)
    tech_df['macd_crossover'] = ((tech_df['macd'] > tech_df['macd_signal']) & 
                                (tech_df['macd'].shift(1) <= tech_df['macd_signal'].shift(1))).astype(int)
    
    # 3. Bollinger Bands with squeeze detection
    print("📊 Calculating Bollinger Bands...")
    def calculate_bollinger_bands_instructor(prices, window=20, num_std=2):
        sma = prices.rolling(window=window).mean()
        std = prices.rolling(window=window).std()
        upper = sma + (num_std * std)
        lower = sma - (num_std * std)
        
        # Additional metrics
        width = (upper - lower) / sma
        position = (prices - lower) / (upper - lower)
        squeeze = width < width.rolling(100).quantile(0.1)  # Bottom 10% of width
        
        return upper, sma, lower, width, position, squeeze
    
    (tech_df['bb_upper'], tech_df['bb_middle'], tech_df['bb_lower'], 
     tech_df['bb_width'], tech_df['bb_position'], tech_df['bb_squeeze']) = calculate_bollinger_bands_instructor(df['close'])
    
    # Band breakouts
    tech_df['bb_breakout_upper'] = (df['close'] > tech_df['bb_upper']).astype(int)
    tech_df['bb_breakout_lower'] = (df['close'] < tech_df['bb_lower']).astype(int)
    
    # 4. Stochastic Oscillator
    print("🎯 Calculating Stochastic...")
    def calculate_stochastic_instructor(high, low, close, k_window=14, d_window=3):
        lowest_low = low.rolling(window=k_window).min()
        highest_high = high.rolling(window=k_window).max()
        k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
        d_percent = k_percent.rolling(window=d_window).mean()
        return k_percent, d_percent
    
    tech_df['stoch_k'], tech_df['stoch_d'] = calculate_stochastic_instructor(df['high'], df['low'], df['close'])
    tech_df['stoch_overbought'] = (tech_df['stoch_k'] > 80).astype(int)
    tech_df['stoch_oversold'] = (tech_df['stoch_k'] < 20).astype(int)
    
    # 5. Williams %R
    print("🔄 Calculating Williams %R...")
    def calculate_williams_r_instructor(high, low, close, window=14):
        highest_high = high.rolling(window=window).max()
        lowest_low = low.rolling(window=window).min()
        williams_r = -100 * ((highest_high - close) / (highest_high - lowest_low))
        return williams_r
    
    tech_df['williams_r'] = calculate_williams_r_instructor(df['high'], df['low'], df['close'])
    
    # 6. Average True Range (ATR)
    print("📏 Calculating ATR...")
    def calculate_atr_instructor(high, low, close, window=14):
        tr1 = high - low
        tr2 = abs(high - close.shift(1))
        tr3 = abs(low - close.shift(1))
        true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = true_range.rolling(window=window).mean()
        return atr
    
    tech_df['atr'] = calculate_atr_instructor(df['high'], df['low'], df['close'])
    tech_df['atr_normalized'] = tech_df['atr'] / df['close']  # ATR as % of price
    
    print(f"✅ Advanced indicators calculated: {len([col for col in tech_df.columns if col not in df.columns])} new features")
    return tech_df

# Apply advanced technical indicators
df_with_indicators = calculate_advanced_indicators_instructor(df_with_basic_features)

# Summary of indicator signals
latest = df_with_indicators.iloc[-1]
print(f"\n📊 CURRENT TECHNICAL SIGNALS:")
print(f"RSI: {latest['rsi']:.1f} ({'Overbought' if latest['rsi'] > 70 else 'Oversold' if latest['rsi'] < 30 else 'Neutral'})")
print(f"MACD: {'Bullish' if latest['macd_bullish'] else 'Bearish'}")
print(f"BB Position: {latest['bb_position']:.3f} ({'Above upper' if latest['bb_position'] > 1 else 'Below lower' if latest['bb_position'] < 0 else 'Within bands'})")
print(f"Stochastic: {latest['stoch_k']:.1f} ({'Overbought' if latest['stoch_k'] > 80 else 'Oversold' if latest['stoch_k'] < 20 else 'Neutral'})")

technical_features = [col for col in df_with_indicators.columns if col not in df_with_basic_features.columns]
print(f"\nTechnical features added: {len(technical_features)}")

## Exercise Solutions

**INSTRUCTOR NOTE**: Complete solutions for all student exercises with detailed explanations and alternative approaches.

### Exercise 2 Solution: Bollinger Bands Implementation

**INSTRUCTOR SOLUTION** with comprehensive analysis and trading applications.

In [ ]:
# INSTRUCTOR SOLUTION - Exercise 2: Complete Bollinger Bands
def bollinger_bands_complete_solution(prices, window=20, num_std=2):
    """
    INSTRUCTOR SOLUTION: Professional Bollinger Bands implementation
    with additional trading metrics and signal generation
    """
    # Core calculations
    middle_band = prices.rolling(window=window).mean()
    rolling_std = prices.rolling(window=window).std()
    upper_band = middle_band + (num_std * rolling_std)
    lower_band = middle_band - (num_std * rolling_std)
    
    # Advanced metrics
    band_width = (upper_band - lower_band) / middle_band
    price_position = (prices - lower_band) / (upper_band - lower_band)
    
    # Trading signals
    squeeze_threshold = band_width.rolling(100).quantile(0.1)
    squeeze = band_width < squeeze_threshold
    
    # Breakout detection
    upper_breakout = prices > upper_band
    lower_breakout = prices < lower_band
    
    # Mean reversion signals
    buy_signal = (price_position < 0.1) & (price_position.shift(1) >= 0.1)
    sell_signal = (price_position > 0.9) & (price_position.shift(1) <= 0.9)
    
    return {
        'upper': upper_band,
        'middle': middle_band, 
        'lower': lower_band,
        'width': band_width,
        'position': price_position,
        'squeeze': squeeze,
        'upper_breakout': upper_breakout,
        'lower_breakout': lower_breakout,
        'buy_signal': buy_signal,
        'sell_signal': sell_signal
    }

# Apply complete solution
bb_solution = bollinger_bands_complete_solution(df_with_indicators['close'])

# Comprehensive visualization
fig, axes = plt.subplots(5, 1, figsize=(15, 20))

# 1. Price with Bollinger Bands and signals
axes[0].plot(df_with_indicators.index, df_with_indicators['close'], 'k-', linewidth=2, label='Close Price')
axes[0].plot(df_with_indicators.index, bb_solution['upper'], 'r--', alpha=0.7, label='Upper Band')
axes[0].plot(df_with_indicators.index, bb_solution['middle'], 'b-', alpha=0.7, label='Middle Band')
axes[0].plot(df_with_indicators.index, bb_solution['lower'], 'g--', alpha=0.7, label='Lower Band')
axes[0].fill_between(df_with_indicators.index, bb_solution['upper'], bb_solution['lower'], alpha=0.1, color='gray')

# Highlight trading signals
buy_dates = df_with_indicators.index[bb_solution['buy_signal']]
sell_dates = df_with_indicators.index[bb_solution['sell_signal']]
if len(buy_dates) > 0:
    axes[0].scatter(buy_dates, df_with_indicators.loc[buy_dates, 'close'], 
                   color='green', marker='^', s=100, label='Buy Signal', zorder=5)
if len(sell_dates) > 0:
    axes[0].scatter(sell_dates, df_with_indicators.loc[sell_dates, 'close'], 
                   color='red', marker='v', s=100, label='Sell Signal', zorder=5)

axes[0].set_title('Bollinger Bands with Trading Signals')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Band Width with squeeze periods
axes[1].plot(df_with_indicators.index, bb_solution['width'], 'purple', linewidth=2)
axes[1].fill_between(df_with_indicators.index, 0, bb_solution['width'], 
                    where=bb_solution['squeeze'], alpha=0.3, color='red', label='Squeeze Periods')
axes[1].set_title('Bollinger Band Width (Volatility Measure)')
axes[1].set_ylabel('Band Width')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 3. Price Position
axes[2].plot(df_with_indicators.index, bb_solution['position'], 'orange', linewidth=2)
axes[2].axhline(y=0.5, color='blue', linestyle='-', alpha=0.5)
axes[2].axhline(y=0.8, color='red', linestyle='--', alpha=0.7)
axes[2].axhline(y=0.2, color='green', linestyle='--', alpha=0.7)
axes[2].fill_between(df_with_indicators.index, 0.8, 1.2, alpha=0.1, color='red', label='Overbought Zone')
axes[2].fill_between(df_with_indicators.index, -0.2, 0.2, alpha=0.1, color='green', label='Oversold Zone')
axes[2].set_title('Price Position Within Bands')
axes[2].set_ylabel('Position (0=Lower, 1=Upper)')
axes[2].set_ylim(-0.2, 1.2)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# 4. Breakout Analysis
breakout_data = pd.DataFrame({
    'Upper Breakouts': bb_solution['upper_breakout'].astype(int),
    'Lower Breakouts': bb_solution['lower_breakout'].astype(int)
}, index=df_with_indicators.index)

axes[3].plot(df_with_indicators.index, breakout_data['Upper Breakouts'], 'r-', alpha=0.7, label='Upper Breakouts')
axes[3].plot(df_with_indicators.index, -breakout_data['Lower Breakouts'], 'g-', alpha=0.7, label='Lower Breakouts')
axes[3].fill_between(df_with_indicators.index, 0, breakout_data['Upper Breakouts'], alpha=0.3, color='red')
axes[3].fill_between(df_with_indicators.index, 0, -breakout_data['Lower Breakouts'], alpha=0.3, color='green')
axes[3].set_title('Band Breakout Events')
axes[3].set_ylabel('Breakout Direction')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

# 5. Distribution Analysis
axes[4].hist(bb_solution['position'].dropna(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[4].axvline(x=0.5, color='blue', linestyle='-', alpha=0.7, label='Middle (50%)')
axes[4].axvline(x=0.8, color='red', linestyle='--', alpha=0.7, label='80% Threshold')
axes[4].axvline(x=0.2, color='green', linestyle='--', alpha=0.7, label='20% Threshold')
axes[4].set_title('Distribution of Price Position')
axes[4].set_xlabel('Position within Bands')
axes[4].set_ylabel('Frequency')
axes[4].legend()
axes[4].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance Analysis
print("🎯 BOLLINGER BANDS PERFORMANCE ANALYSIS")
print("=" * 50)
print(f"Upper breakouts: {bb_solution['upper_breakout'].sum()} ({(bb_solution['upper_breakout'].sum()/len(df_with_indicators))*100:.2f}%)")
print(f"Lower breakouts: {bb_solution['lower_breakout'].sum()} ({(bb_solution['lower_breakout'].sum()/len(df_with_indicators))*100:.2f}%)")
print(f"Squeeze periods: {bb_solution['squeeze'].sum()} days ({(bb_solution['squeeze'].sum()/len(df_with_indicators))*100:.2f}%)")
print(f"Buy signals: {bb_solution['buy_signal'].sum()}")
print(f"Sell signals: {bb_solution['sell_signal'].sum()}")
print(f"Average band width: {bb_solution['width'].mean():.4f}")

# Position statistics
pos_stats = bb_solution['position'].describe()
print(f"\nPrice Position Statistics:")
for stat, value in pos_stats.items():
    print(f"  {stat}: {value:.4f}")

### Exercise 3 Solution: PCA vs Feature Importance

**INSTRUCTOR SOLUTION** with comprehensive comparison and practical recommendations.

In [ ]:
# INSTRUCTOR SOLUTION - Exercise 3: Complete PCA vs Feature Importance Analysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

def complete_pca_vs_importance_analysis(df):
    """
    INSTRUCTOR SOLUTION: Comprehensive PCA vs Feature Importance comparison
    with model performance evaluation
    """
    print("🔬 COMPLETE PCA vs FEATURE IMPORTANCE ANALYSIS")
    print("=" * 60)
    
    # Prepare comprehensive feature set
    feature_columns = [col for col in df.columns if col not in ['open', 'high', 'low', 'close', 'volume']]
    features_df = df[feature_columns].dropna()
    
    print(f"Feature matrix shape: {features_df.shape}")
    print(f"Features analyzed: {len(feature_columns)}")
    
    # Target variable for importance ranking
    target = features_df['returns'].shift(-1).dropna()
    X = features_df.iloc[:-1]  # Remove last row to match target
    
    # 1. COMPREHENSIVE PCA ANALYSIS
    print(f"\n{'='*20} PCA ANALYSIS {'='*20}")
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Fit PCA
    pca = PCA()
    pca_components = pca.fit_transform(X_scaled)
    
    # Explained variance analysis
    explained_var = pca.explained_variance_ratio_
    cumsum_var = np.cumsum(explained_var)
    
    # Find components for different variance thresholds
    components_80 = np.argmax(cumsum_var >= 0.80) + 1
    components_90 = np.argmax(cumsum_var >= 0.90) + 1
    components_95 = np.argmax(cumsum_var >= 0.95) + 1
    
    print(f"Components for 80% variance: {components_80}")
    print(f"Components for 90% variance: {components_90}")
    print(f"Components for 95% variance: {components_95}")
    
    # Get detailed loadings
    n_components_detail = min(5, len(feature_columns))
    loadings = pd.DataFrame(
        pca.components_[:n_components_detail].T,
        columns=[f'PC{i+1}' for i in range(n_components_detail)],
        index=X.columns
    )
    
    # 2. RANDOM FOREST FEATURE IMPORTANCE
    print(f"\n{'='*15} RANDOM FOREST ANALYSIS {'='*15}")
    
    # Multiple RF models with different parameters
    rf_results = {}
    for n_est, max_d in [(50, 5), (100, 10), (200, 15)]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=max_d, 
                                 random_state=42, n_jobs=-1)
        rf.fit(X, target)
        
        importance_df = pd.DataFrame({
            'feature': X.columns,
            'importance': rf.feature_importances_
        }).sort_values('importance', ascending=False)
        
        rf_results[f'RF_{n_est}_{max_d}'] = importance_df
    
    # Use the middle configuration for main analysis
    main_rf_result = rf_results['RF_100_10']
    
    # 3. CORRELATION ANALYSIS
    print(f"\n{'='*15} CORRELATION ANALYSIS {'='*15}")
    
    # Calculate correlations between different rankings
    correlations = {}
    
    for pc in [f'PC{i+1}' for i in range(min(3, n_components_detail))]:
        pc_loadings_abs = loadings[pc].abs().sort_values(ascending=False)
        
        # Align with RF importance
        rf_aligned = main_rf_result.set_index('feature')['importance']
        common_features = pc_loadings_abs.index.intersection(rf_aligned.index)
        
        if len(common_features) > 0:
            corr = pc_loadings_abs[common_features].corr(rf_aligned[common_features])
            correlations[f'{pc}_vs_RF'] = corr
    
    # 4. MODEL PERFORMANCE COMPARISON
    print(f"\n{'='*15} MODEL PERFORMANCE {'='*15}")
    
    # Split data for evaluation
    split_idx = int(0.8 * len(X))
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = target.iloc[:split_idx], target.iloc[split_idx:]
    
    # Scale for PCA
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    performance_results = {}
    
    # Original features
    rf_orig = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf_orig.fit(X_train, y_train)
    y_pred_orig = rf_orig.predict(X_test)
    performance_results['Original Features'] = {
        'R2': r2_score(y_test, y_pred_orig),
        'MSE': mean_squared_error(y_test, y_pred_orig),
        'Features': len(X.columns)
    }
    
    # PCA features
    for n_comp in [components_80, components_90, components_95]:
        pca_model = PCA(n_components=n_comp)
        X_train_pca = pca_model.fit_transform(X_train_scaled)
        X_test_pca = pca_model.transform(X_test_scaled)
        
        rf_pca = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
        rf_pca.fit(X_train_pca, y_train)
        y_pred_pca = rf_pca.predict(X_test_pca)
        
        var_explained = sum(pca_model.explained_variance_ratio_)
        performance_results[f'PCA_{n_comp}_components'] = {
            'R2': r2_score(y_test, y_pred_pca),
            'MSE': mean_squared_error(y_test, y_pred_pca),
            'Features': n_comp,
            'Variance_Explained': var_explained
        }
    
    # Top RF features
    for top_n in [10, 15, 20]:
        if top_n <= len(main_rf_result):
            top_features = main_rf_result.head(top_n)['feature'].tolist()
            X_train_top = X_train[top_features]
            X_test_top = X_test[top_features]
            
            rf_top = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
            rf_top.fit(X_train_top, y_train)
            y_pred_top = rf_top.predict(X_test_top)
            
            performance_results[f'Top_{top_n}_RF_features'] = {
                'R2': r2_score(y_test, y_pred_top),
                'MSE': mean_squared_error(y_test, y_pred_top),
                'Features': top_n
            }
    
    return {
        'pca_results': {
            'explained_variance': explained_var,
            'cumsum_variance': cumsum_var,
            'loadings': loadings,
            'components_80': components_80,
            'components_90': components_90,
            'components_95': components_95
        },
        'rf_results': rf_results,
        'correlations': correlations,
        'performance': performance_results,
        'feature_columns': feature_columns
    }

# Execute complete analysis
analysis_results = complete_pca_vs_importance_analysis(df_with_indicators)

# Create comprehensive visualization
fig, axes = plt.subplots(3, 2, figsize=(18, 15))

# 1. PCA Explained Variance
pca_res = analysis_results['pca_results']
axes[0, 0].bar(range(1, min(21, len(pca_res['explained_variance'])) + 1), 
               pca_res['explained_variance'][:20], alpha=0.7, color='steelblue')
axes[0, 0].plot(range(1, min(21, len(pca_res['cumsum_variance'])) + 1), 
                pca_res['cumsum_variance'][:20], 'ro-', markersize=4)
axes[0, 0].axhline(y=0.8, color='green', linestyle='--', alpha=0.7, label='80%')
axes[0, 0].axhline(y=0.9, color='orange', linestyle='--', alpha=0.7, label='90%')
axes[0, 0].axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95%')
axes[0, 0].set_xlabel('Principal Component')
axes[0, 0].set_ylabel('Explained Variance Ratio')
axes[0, 0].set_title('PCA Explained Variance')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Feature Importance
main_rf = analysis_results['rf_results']['RF_100_10']
top_features = main_rf.head(15)
axes[0, 1].barh(range(len(top_features)), top_features['importance'], color='forestgreen', alpha=0.7)
axes[0, 1].set_yticks(range(len(top_features)))
axes[0, 1].set_yticklabels(top_features['feature'])
axes[0, 1].set_xlabel('Importance Score')
axes[0, 1].set_title('Random Forest Feature Importance (Top 15)')
axes[0, 1].grid(True, alpha=0.3)

# 3. PCA Loadings Heatmap
sns.heatmap(pca_res['loadings'].T, annot=False, cmap='RdBu_r', center=0, 
            ax=axes[1, 0], cbar_kws={'label': 'Loading'})
axes[1, 0].set_title('PCA Loadings Matrix')
axes[1, 0].set_xlabel('Features')

# 4. Performance Comparison
perf_data = analysis_results['performance']
methods = list(perf_data.keys())
r2_scores = [perf_data[method]['R2'] for method in methods]
feature_counts = [perf_data[method]['Features'] for method in methods]

scatter = axes[1, 1].scatter(feature_counts, r2_scores, s=100, alpha=0.7, c=range(len(methods)), cmap='viridis')
for i, method in enumerate(methods):
    axes[1, 1].annotate(method, (feature_counts[i], r2_scores[i]), 
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[1, 1].set_xlabel('Number of Features')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_title('Model Performance vs Feature Count')
axes[1, 1].grid(True, alpha=0.3)

# 5. Correlation Analysis
if analysis_results['correlations']:
    corr_data = analysis_results['correlations']
    corr_names = list(corr_data.keys())
    corr_values = list(corr_data.values())
    
    axes[2, 0].bar(corr_names, corr_values, alpha=0.7, color='purple')
    axes[2, 0].set_ylabel('Correlation Coefficient')
    axes[2, 0].set_title('PCA Loadings vs RF Importance Correlation')
    axes[2, 0].tick_params(axis='x', rotation=45)
    axes[2, 0].grid(True, alpha=0.3)

# 6. Feature Reduction Efficiency
methods_subset = [m for m in methods if 'PCA' in m or 'Top_' in m]
efficiency_data = []
for method in methods_subset:
    perf = perf_data[method]
    efficiency = perf['R2'] / perf['Features']  # R² per feature
    efficiency_data.append((method, efficiency, perf['Features'], perf['R2']))

efficiency_data.sort(key=lambda x: x[1], reverse=True)
method_names = [x[0] for x in efficiency_data]
efficiency_scores = [x[1] for x in efficiency_data]

axes[2, 1].bar(range(len(method_names)), efficiency_scores, alpha=0.7, color='coral')
axes[2, 1].set_xticks(range(len(method_names)))
axes[2, 1].set_xticklabels(method_names, rotation=45, ha='right')
axes[2, 1].set_ylabel('R² per Feature')
axes[2, 1].set_title('Feature Reduction Efficiency')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Detailed Results Summary
print(f"\n{'='*20} ANALYSIS SUMMARY {'='*20}")
print(f"Total features analyzed: {len(analysis_results['feature_columns'])}")
print(f"PCA components for 95% variance: {pca_res['components_95']}")
print(f"Variance reduction: {len(analysis_results['feature_columns'])} → {pca_res['components_95']} features")

print(f"\nTop 5 most important features (RF):")
for i, row in main_rf.head(5).iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

print(f"\nModel Performance Summary:")
best_performance = max(perf_data.items(), key=lambda x: x[1]['R2'])
print(f"Best R²: {best_performance[1]['R2']:.4f} ({best_performance[0]})")

most_efficient = max([(k, v['R2']/v['Features']) for k, v in perf_data.items()], key=lambda x: x[1])
print(f"Most efficient: {most_efficient[1]:.6f} R²/feature ({most_efficient[0]})")

print(f"\nRECOMMENDATIONS:")
if pca_res['components_95'] < len(analysis_results['feature_columns']) * 0.5:
    print(f"✅ PCA recommended: Reduces {len(analysis_results['feature_columns'])} features to {pca_res['components_95']} with minimal information loss")
else:
    print(f"⚠️  Feature selection may be better: PCA doesn't provide significant dimensionality reduction")

print(f"🎯 Exercise 3 Complete: Comprehensive PCA vs Feature Importance analysis finished!")

## Week 2 Challenge: Volatility Regime Detection

**INSTRUCTOR NOTE**: This advanced challenge introduces regime-switching models and volatility forecasting - preparation for Week 3 content.

In [ ]:
# INSTRUCTOR SOLUTION - Advanced Challenge: Volatility Regime Detection
def volatility_regime_detection_instructor(df, lookback=30):
    """
    INSTRUCTOR SOLUTION: Advanced volatility regime detection using multiple methods
    """
    print("🎯 ADVANCED CHALLENGE: VOLATILITY REGIME DETECTION")
    print("=" * 60)
    
    # Calculate various volatility measures
    returns = df['close'].pct_change()
    
    # 1. Rolling volatility (multiple windows)
    vol_5d = returns.rolling(5).std() * np.sqrt(252)
    vol_20d = returns.rolling(20).std() * np.sqrt(252)
    vol_60d = returns.rolling(60).std() * np.sqrt(252)
    
    # 2. GARCH-style volatility (simplified)
    vol_garch = returns.ewm(alpha=0.06).std() * np.sqrt(252)
    
    # 3. Realized volatility using OHLC
    realized_vol = np.log(df['high'] / df['low']) * np.sqrt(252)
    
    # 4. Regime detection using quantiles
    vol_primary = vol_20d  # Use 20-day as primary measure
    
    # Define regimes based on rolling quantiles
    low_threshold = vol_primary.rolling(lookback*5).quantile(0.33)
    high_threshold = vol_primary.rolling(lookback*5).quantile(0.67)
    
    # Create regime indicators
    regimes = pd.Series(index=df.index, dtype=int)
    regimes = regimes.fillna(1)  # Default to medium volatility
    
    regimes[vol_primary <= low_threshold] = 0  # Low volatility
    regimes[vol_primary >= high_threshold] = 2  # High volatility
    
    # 5. Regime transition analysis
    regime_changes = regimes.diff().abs() > 0
    transitions = regimes[regime_changes]
    
    # 6. Persistence analysis
    regime_runs = []
    current_regime = regimes.iloc[0] if not pd.isna(regimes.iloc[0]) else 1
    run_length = 1
    
    for i in range(1, len(regimes)):
        if regimes.iloc[i] == current_regime:
            run_length += 1
        else:
            regime_runs.append((current_regime, run_length))
            current_regime = regimes.iloc[i]
            run_length = 1
    
    # Add final run
    regime_runs.append((current_regime, run_length))
    
    # Calculate average persistence
    avg_persistence = {}
    for regime in [0, 1, 2]:
        regime_lengths = [length for reg, length in regime_runs if reg == regime]
        avg_persistence[regime] = np.mean(regime_lengths) if regime_lengths else 0
    
    return {
        'volatilities': {
            'vol_5d': vol_5d,
            'vol_20d': vol_20d,
            'vol_60d': vol_60d,
            'vol_garch': vol_garch,
            'realized_vol': realized_vol
        },
        'regimes': regimes,
        'thresholds': {
            'low': low_threshold,
            'high': high_threshold
        },
        'transitions': transitions,
        'persistence': avg_persistence,
        'regime_runs': regime_runs
    }

# Apply volatility regime detection
regime_results = volatility_regime_detection_instructor(df_with_indicators)

# Comprehensive visualization
fig, axes = plt.subplots(4, 1, figsize=(15, 16))

# 1. Price with regime background
regimes = regime_results['regimes']
axes[0].plot(df_with_indicators.index, df_with_indicators['close'], 'k-', linewidth=2, label='Close Price')

# Color background by regime
for regime, color, alpha, label in [(0, 'green', 0.2, 'Low Vol'), (1, 'yellow', 0.1, 'Medium Vol'), (2, 'red', 0.2, 'High Vol')]:
    regime_mask = regimes == regime
    if regime_mask.any():
        # Create continuous segments for background
        regime_starts = regime_mask & (~regime_mask.shift(1).fillna(False))
        regime_ends = regime_mask & (~regime_mask.shift(-1).fillna(False))
        
        for start_idx in df_with_indicators.index[regime_starts]:
            # Find corresponding end
            end_candidates = df_with_indicators.index[regime_ends & (df_with_indicators.index >= start_idx)]
            if len(end_candidates) > 0:
                end_idx = end_candidates[0]
                axes[0].axvspan(start_idx, end_idx, alpha=alpha, color=color, label=label if start_idx == df_with_indicators.index[regime_starts][0] else "")

axes[0].set_title('Stock Price with Volatility Regimes')
axes[0].set_ylabel('Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Multiple volatility measures
vols = regime_results['volatilities']
axes[1].plot(df_with_indicators.index, vols['vol_5d'], label='5-day Vol', alpha=0.7)
axes[1].plot(df_with_indicators.index, vols['vol_20d'], label='20-day Vol', linewidth=2)
axes[1].plot(df_with_indicators.index, vols['vol_60d'], label='60-day Vol', alpha=0.7)
axes[1].plot(df_with_indicators.index, vols['vol_garch'], label='GARCH-style Vol', linestyle='--')

# Add regime thresholds
thresholds = regime_results['thresholds']
axes[1].plot(df_with_indicators.index, thresholds['low'], 'g--', alpha=0.7, label='Low Threshold')
axes[1].plot(df_with_indicators.index, thresholds['high'], 'r--', alpha=0.7, label='High Threshold')

axes[1].set_title('Volatility Measures and Regime Thresholds')
axes[1].set_ylabel('Annualized Volatility')
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].grid(True, alpha=0.3)

# 3. Regime time series
regime_colors = {0: 'green', 1: 'yellow', 2: 'red'}
regime_labels = {0: 'Low Volatility', 1: 'Medium Volatility', 2: 'High Volatility'}

for regime in [0, 1, 2]:
    regime_mask = regimes == regime
    axes[2].scatter(df_with_indicators.index[regime_mask], [regime]*regime_mask.sum(), 
                   c=regime_colors[regime], alpha=0.6, s=20, label=regime_labels[regime])

axes[2].set_title('Volatility Regime Classification Over Time')
axes[2].set_ylabel('Regime')
axes[2].set_yticks([0, 1, 2])
axes[2].set_yticklabels(['Low', 'Medium', 'High'])
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# 4. Regime statistics
regime_counts = regimes.value_counts().sort_index()
regime_percentages = (regime_counts / len(regimes)) * 100

bars = axes[3].bar(regime_counts.index, regime_percentages, 
                  color=[regime_colors[i] for i in regime_counts.index], alpha=0.7)
axes[3].set_title('Volatility Regime Distribution')
axes[3].set_xlabel('Regime')
axes[3].set_ylabel('Percentage of Time')
axes[3].set_xticks([0, 1, 2])
axes[3].set_xticklabels(['Low Vol', 'Medium Vol', 'High Vol'])

# Add percentage labels on bars
for bar, pct in zip(bars, regime_percentages):
    height = bar.get_height()
    axes[3].text(bar.get_x() + bar.get_width()/2., height + 0.5, f'{pct:.1f}%',
                ha='center', va='bottom')

axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Detailed regime analysis
print(f"\n🔍 REGIME ANALYSIS RESULTS:")
print(f"{'='*40}")

persistence = regime_results['persistence']
print(f"Average Regime Persistence (days):")
print(f"  Low Volatility:    {persistence[0]:.1f}")
print(f"  Medium Volatility: {persistence[1]:.1f}")  
print(f"  High Volatility:   {persistence[2]:.1f}")

print(f"\nRegime Distribution:")
for regime, count in regime_counts.items():
    pct = (count / len(regimes)) * 100
    print(f"  {regime_labels[regime]}: {count} days ({pct:.1f}%)")

print(f"\nRegime Transitions: {len(regime_results['transitions'])} changes")

# Current regime
current_regime = regimes.iloc[-1] if not pd.isna(regimes.iloc[-1]) else 1
current_vol = vols['vol_20d'].iloc[-1]
print(f"\nCurrent Status:")
print(f"  Regime: {regime_labels[current_regime]}")
print(f"  20-day Volatility: {current_vol:.1f}%")

print(f"\n🎯 Advanced Challenge Complete: Volatility regime detection implemented!")

## Summary and Next Steps

**INSTRUCTOR NOTE**: Comprehensive review of Week 2 concepts and preparation for Week 3.

### Key Concepts Covered:
1. **Data Cleaning**: Professional-grade data validation and preprocessing
2. **Basic Features**: Returns, moving averages, volatility measures
3. **Technical Indicators**: RSI, MACD, Bollinger Bands, Stochastic, Williams %R, ATR
4. **Advanced Analysis**: PCA vs Feature Importance comparison
5. **Regime Detection**: Volatility regime classification and persistence analysis

### Student Assessment Checklist:
- [ ] Can implement robust data loading with error handling
- [ ] Understands different return calculations (simple vs log)
- [ ] Can calculate and interpret technical indicators
- [ ] Understands when to use PCA vs feature selection
- [ ] Can detect and analyze volatility regimes

### Week 3 Preparation:
- Model selection and hyperparameter tuning
- Cross-validation for time series
- Advanced ensemble methods
- Risk metrics and portfolio optimization